In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time 
import json

In [2]:
#%% Connect to Zillow

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/',
    'x-requested-with': 'XMLHttpRequest'
}


In [3]:
def request_zillow(City,input_page):
    with requests.Session() as s:
        #City = 'austin/' #*****change this city to what you want*****
        #url = 'https://www.zillow.com/homes/sold/'+City+ input_page
        
        
        url = 'https://www.zillow.com/homes/sold/Columbus,-IN_rb/' + input_page
        r = s.get(url, headers=req_headers)
        return r

In [4]:
df_total = pd.DataFrame()
for w in range(20): # 20 pages which is about 800 houses
    input_page = str(w+1)+'_p/'
    time.sleep(2) # Cannot request more than 2 pages per sec or CAPTCHA thing will show up
    r = request_zillow('',input_page)
    soup = BeautifulSoup(r.content, 'html.parser')
    #had to look into the webscape object and figure out how to pull the data
    zillow_houses= soup.find_all("script", {'data-zrr-shared-data-key': 'mobileSearchPageStore'})
    zillow_houses = str(zillow_houses);
    zillow_houses = zillow_houses.replace('[<script data-zrr-shared-data-key="mobileSearchPageStore" type="application/json"><!--','')
    zillow_houses = zillow_houses.replace('--></script>]','')
    zillowhouses_json = json.loads(zillow_houses)
    zillowhouses_json = zillowhouses_json.get("cat1").get("searchResults").get("listResults")
    df = pd.DataFrame.from_dict(zillowhouses_json)
    df_total = df_total.append(df, ignore_index=True)

In [5]:
pd.set_option('max_columns', None)
df_total.head(5)

,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,soldPrice,unformattedPrice,address,addressStreet,addressCity,addressState,addressZipcode,isUndisclosedAddress,beds,baths,area,latLong,isZillowOwned,variableData,badgeInfo,hdpData,isSaved,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,info1String,brokerName,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,lotAreaString,info2String,info6String,streetViewMetadataURL,streetViewURL,builderName
0,85096619,85096619,None,https://photos.zillowstatic.com/fp/cb84e97172e...,True,https://www.zillow.com/homedetails/1226-Pine-H...,SOLD,Sold,$,"$342,000",342000,"1226 Pine Hill Dr, Columbus, IN 47201",1226 Pine Hill Dr,Columbus,IN,47201,False,4.0,3.0,2958.0,"{'latitude': 39.18583, 'longitude': -85.96752}",False,"{'type': 'RECENTLY_SOLD', 'text': 'Sold 01/07/...",None,"{'homeInfo': {'zpid': 85096619, 'streetAddress...",False,False,False,RecentlySold,RecentChange,329900.0,False,False,False,False,MLS ID #21827695,RE/MAX Real Estate Prof,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN
1,85100166,85100166,None,https://photos.zillowstatic.com/fp/a5845804dc7...,True,https://www.zillow.com/homedetails/1503-Ruddic...,SOLD,Sold,$,"$82,500",82500,"1503 Ruddick Ave, Columbus, IN 47201",1503 Ruddick Ave,Columbus,IN,47201,False,2.0,1.0,1284.0,"{'latitude': 39.21405, 'longitude': -85.90995}",False,"{'type': 'RECENTLY_SOLD', 'text': 'Sold 01/07/...",None,"{'homeInfo': {'zpid': 85100166, 'streetAddress...",False,False,False,RecentlySold,Unknown Listed By,126400.0,False,False,False,False,MLS ID #21828236,Schuler Bauer Real Estate Powe,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2067520593,2067520593,None,https://photos.zillowstatic.com/fp/9684b7b9135...,True,https://www.zillow.com/homedetails/5128-Montpe...,SOLD,Sold,$,"$260,250",260250,"5128 Montpelier Ct, Columbus, IN 47203",5128 Montpelier Ct,Columbus,IN,47203,False,3.0,2.0,1979.0,"{'latitude': 39.230843, 'longitude': -85.86081}",False,"{'type': 'RECENTLY_SOLD', 'text': 'Sold 01/07/...",None,"{'homeInfo': {'zpid': 2067520593, 'streetAddre...",False,False,False,RecentlySold,RecentChange,NaN,False,False,False,False,MLS ID #21825088,RE/MAX Real Estate Prof,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,112013093,112013093,None,https://photos.zillowstatic.com/fp/279dc16808f...,True,https://www.zillow.com/homedetails/Columbus-IN...,SOLD,Sold,$,"$245,000",245000,"(undisclosed Address), Columbus, IN 47201",(undisclosed Address),Columbus,IN,47201,True,4.0,3.0,2110.0,{},False,"{'type': 'RECENTLY_SOLD', 'text': 'Sold 01/06/...",None,"{'homeInfo': {'zpid': 112013093, 'streetAddres...",False,False,False,RecentlySold,RecentChange,244900.0,False,False,True,False,MLS ID #21828970,"HOME Real Estate Services, LLC",True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN
4,85094806,85094806,None,https://photos.zillowstatic.com/fp/8ab603b0ac7...,True,https://www.zillow.com/homedetails/3486-Washin...,SOLD,Sold,$,"$162,000",162000,"3486 Washington St, Columbus, IN 47203",3486 Washington St,Columbus,IN,47203,False,3.0,2.0,1026.0,"{'latitude': 39.236782, 'longitude': -85.92159}",False,"{'type': 'RECENTLY_SOLD', 'text': 'Sold 01/05/...",None,"{'homeInfo': {'zpid': 85094806, 'streetAddress...",False,False,False,RecentlySold,RecentChange,148000.0,False,False,False,False,MLS ID #21831359,RE/MAX Real Estate Prof,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
info = df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zpid                        800 non-null    object 
 1   id                          800 non-null    object 
 2   providerListingId           13 non-null     object 
 3   imgSrc                      800 non-null    object 
 4   hasImage                    794 non-null    object 
 5   detailUrl                   800 non-null    object 
 6   statusType                  800 non-null    object 
 7   statusText                  800 non-null    object 
 8   countryCurrency             800 non-null    object 
 9   soldPrice                   800 non-null    object 
 10  unformattedPrice            800 non-null    int64  
 11  address                     800 non-null    object 
 12  addressStreet               800 non-null    object 
 13  addressCity                 800 non

When we are exploring the dataframe that we are not familar with, I like to take a look at the first 5 rows of the data with all the elements in the column to get a basic understanding of the data. And also use df.info() to explore the type of the elements in each column.

In this case, we see this Dataframe has 46 columns and various object type.

There are some interesting data in this dataset such as sold prices, beds, baths, area, location, Broker Compannies, Zestimate and etc.

We also see this dataframe has 3 dict objects i.e latLongs, variableData (has sold date), hdpData. Therefore let's extract those data before we do any data cleaning.


In [7]:
latLong_data = df_total.latLong.astype('str').apply(eval).apply(pd.Series)
variable_data= df_total.variableData.astype('str').apply(eval).apply(pd.Series)
hdp_data = df_total.hdpData.astype('str').apply(eval).apply(pd.Series)

df_total = pd.concat([df_total, latLong_data,variable_data,hdp_data], axis=1, join='inner')
df_total = df_total.drop(columns=['latLong','variableData','hdpData'])

In [8]:
df_total.head(1)

,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,soldPrice,unformattedPrice,address,addressStreet,addressCity,addressState,addressZipcode,isUndisclosedAddress,beds,baths,area,isZillowOwned,badgeInfo,isSaved,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,info1String,brokerName,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,lotAreaString,info2String,info6String,streetViewMetadataURL,streetViewURL,builderName,latitude,longitude,type,text,homeInfo
0,85096619,85096619,None,https://photos.zillowstatic.com/fp/cb84e97172e...,True,https://www.zillow.com/homedetails/1226-Pine-H...,SOLD,Sold,$,"$342,000",342000,"1226 Pine Hill Dr, Columbus, IN 47201",1226 Pine Hill Dr,Columbus,IN,47201,False,4.0,3.0,2958.0,False,None,False,False,False,RecentlySold,RecentChange,329900.0,False,False,False,False,MLS ID #21827695,RE/MAX Real Estate Prof,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN,39.18583,-85.96752,RECENTLY_SOLD,Sold 01/07/2022,"{'zpid': 85096619, 'streetAddress': '1226 Pine..."


In [9]:
homeInfo_data = df_total.homeInfo.apply(pd.Series)

In [10]:
df_total = pd.concat([df_total, homeInfo_data], axis=1, join='inner').drop(columns=['homeInfo'])

There are lots of duplicate Column Names. Let's rename them so it makes it easier for us to process the data later

In [11]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] = 0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

df_total = df_total.rename(columns=renamer())

In [12]:
df_total.head(1)

,zpid,id,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,countryCurrency,soldPrice,unformattedPrice,address,addressStreet,addressCity,addressState,addressZipcode,isUndisclosedAddress,beds,baths,area,isZillowOwned,badgeInfo,isSaved,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,zestimate,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,info1String,brokerName,hasAdditionalAttributions,isFeaturedListing,availabilityDate,list,relaxed,lotAreaString,info2String,info6String,streetViewMetadataURL,streetViewURL,builderName,latitude,longitude,type,text,zpid_1,streetAddress,zipcode,city,state,latitude_1,longitude_1,price,dateSold,bathrooms,bedrooms,livingArea,homeType,homeStatus,daysOnZillow,isFeatured,shouldHighlight,zestimate_1,rentZestimate,listing_sub_type,isUnmappable,isPreforeclosureAuction,homeStatusForHDP,priceForHDP,isNonOwnerOccupied,isPremierBuilder,isZillowOwned_1,currency,country,taxAssessedValue,lotAreaValue,lotAreaUnit,videoCount,unit,isRentalWithBasePrice,newConstructionType,datePriceChanged,priceReduction,priceChange
0,85096619,85096619,None,https://photos.zillowstatic.com/fp/cb84e97172e...,True,https://www.zillow.com/homedetails/1226-Pine-H...,SOLD,Sold,$,"$342,000",342000,"1226 Pine Hill Dr, Columbus, IN 47201",1226 Pine Hill Dr,Columbus,IN,47201,False,4.0,3.0,2958.0,False,None,False,False,False,RecentlySold,RecentChange,329900.0,False,False,False,False,MLS ID #21827695,RE/MAX Real Estate Prof,True,False,None,True,False,NaN,NaN,NaN,NaN,NaN,NaN,39.18583,-85.96752,RECENTLY_SOLD,Sold 01/07/2022,85096619,1226 Pine Hill Dr,47201,Columbus,IN,39.18583,-85.96752,342000.0,1641542400000,3.0,4.0,2958.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,329900.0,1714.0,{},False,False,RECENTLY_SOLD,342000.0,True,False,False,USD,USA,210200.0,0.4095,acres,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Cleaning involves lots of effort. So first let's ask ourselves, what questions are we trying to answer?

1. For this zillow dataset, there are couple of things interest to me such as looking for correlation between sold prices vs zestimate prices, prices vs beds/baths/area 
2. How many houses are sold per each unique Borker?
3. Since there are location data, I'm interested to plot the location of the houses and see if certain neighborhood is more expensive compare others. 
4. Can we further pull information of each houses information from the house url?

It seems like we would need prices, zestimate price, bed/bath/area, date solded, lat, long, brokername and url

let's go to work!

In [13]:
df_clean = df_total[['detailUrl', 'unformattedPrice','beds','baths','area','zestimate','brokerName','latitude','longitude','text']]

In [14]:
df_clean.head(1)

,detailUrl,unformattedPrice,beds,baths,area,zestimate,brokerName,latitude,longitude,text
0,https://www.zillow.com/homedetails/1226-Pine-H...,342000,4.0,3.0,2958.0,329900.0,RE/MAX Real Estate Prof,39.18583,-85.96752,Sold 01/07/2022


Seems like the date needs to do some processing

In [15]:
from datetime import datetime

In [16]:
def process_text(x):
    x = x.replace("Sold ","")
    x = datetime.strptime(x, "%m/%d/%Y")
    return x

In [17]:
df_clean['date'] = df_clean['text'].apply(lambda x: process_text(x))

<ipython-input-17-64fe9746ab47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date'] = df_clean['text'].apply(lambda x: process_text(x))


In [18]:
df_clean = df_clean.drop(columns = ['text'])

Save the Data to Excel for Visulization later

In [20]:
df_clean.to_csv('Zillow_Clean_Data.csv',index=False)

In [1]:
def request_zillow_url(url):
    with requests.Session() as s:
        r = s.get(url, headers=req_headers)
        return r